In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd

In [14]:
config = configparser.RawConfigParser()
config.read(r'C:\Users\m088378\Documents\myKeyVariables.cfg')   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False
outDir = envVars['pointsdownloaded']

conn = ezomero.connect(user, password, 'Wang.C_Lab', host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	OVCA_TMA22:	408
	0004_VisCyt_CW_09132023:	859


In [15]:
allSamps = ezomero.get_dataset_ids(conn, project=408)
print("There are {} datasets to investigate.".format(len(allSamps)))

There are 5 datasets to investigate.


In [16]:
## Image Desc
## https://docs.openmicroscopy.org/omero/5.4.1/developers/PythonBlitzGateway.html
# https://downloads.openmicroscopy.org/omero/5.4.10/api/python/omero/omero.gateway.html#omero.gateway._ImageWrapper
fout = open('log_omero_points.txt', 'w')
for sm in allSamps:
    imgs = ezomero.get_image_ids(conn, dataset=sm)
    for im in imgs:
        imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
        if(imgDesc.getROICount(shapeType="Point") > 0):
            fout.write(imgDesc.getName()+'\n')
            shName = imgDesc.getName().replace(".ome.tiff",'')
            imgPointsIDs = ezomero.get_roi_ids(conn, im)
            print("{} has {}".format(imgDesc.getName().replace('\\.ome\\.tiff',''), len(imgPointsIDs)))
            fout.write("{} ROIs\n".format(len(imgPointsIDs)))
            allThesePoints = []
            for i in imgPointsIDs:
                for ppI in ezomero.get_shape_ids(conn, i):
                    try:
                        pt, fill, border, width = ezomero.get_shape(conn, ppI)
                        allThesePoints.append([pt.x, pt.y, pt.label,pt.label, fill, border, width])
                    except:
                        print("   BAD point ID : "+str(i))
            #pprint(allThesePoints)
            pointTable = pd.DataFrame(allThesePoints)
            pointTable.columns = ["x","y","class", "name","fill","border","width"]
            cntTbl = pointTable['class'].value_counts().to_frame()
            noms = cntTbl.index.to_list()
            cnts = cntTbl.values
            for n in range(len(noms)):
                fout.write("{}\t{}\n".format(noms[n],cnts[n][0]))
            fout.write("\n")
            pointTable.to_csv(os.path.join(outDir,shName+'-points.tsv'), index=False, sep="\t")


OVCA_TMA22_region_110.ome.tiff has 179 ROIs
OVCA_TMA22_region_107.ome.tiff has 174 ROIs
OVCA_TMA22_region_064.ome.tiff has 166 ROIs
OVCA_TMA22_region_055.ome.tiff has 174 ROIs
OVCA_TMA22_region_135.ome.tiff has 103 ROIs
OVCA_TMA22_region_046.ome.tiff has 43 ROIs
OVCA_TMA22_region_065.ome.tiff has 181 ROIs
OVCA_TMA22_region_195.ome.tiff has 238 ROIs
OVCA_TMA22_region_172.ome.tiff has 191 ROIs
OVCA_TMA22_region_313.ome.tiff has 230 ROIs
OVCA_TMA22_region_070.ome.tiff has 248 ROIs
OVCA_TMA22_region_039.ome.tiff has 95 ROIs
OVCA_TMA22_region_137.ome.tiff has 189 ROIs
OVCA_TMA22_region_060.ome.tiff has 104 ROIs
OVCA_TMA22_region_145.ome.tiff has 183 ROIs
OVCA_TMA22_region_062.ome.tiff has 169 ROIs
OVCA_TMA22_region_030.ome.tiff has 288 ROIs
OVCA_TMA22_region_087.ome.tiff has 301 ROIs
OVCA_TMA22_region_093.ome.tiff has 254 ROIs
OVCA_TMA22_region_144.ome.tiff has 156 ROIs
OVCA_TMA22_region_049.ome.tiff has 168 ROIs
OVCA_TMA22_region_179.ome.tiff has 76 ROIs
OVCA_TMA22_region_132.ome.tiff has 

In [17]:
fout.close()